# Preprocessing on commentary

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

In [126]:
# text = data['Comment'].str.split(',')     #splitting the comment based on commas
# data['Comment'] = text.str.slice(start=2) #removing player names and the ball_event
# players = text.str.get(0).str.split(' to ') #getting the first element after splitting and splitting that based on 'to'
# data['Bowler'] = players.str.get(0)       #Bowler name is the first element after splitting players
# data['Batsman'] = players.str.get(1)      #Batsman name is the second element after splitting players
# data[['Ball_Event','Comment']]            # printing two columns together

In [3]:
df = pd.read_csv("/Users/murali/code/gyandata/table154.csv")
df.loc[0, "Comment"]

"Ali to Anwar Ali, OUT, tossed up, driven, plucked low at short cover, and England have sealed the series! Or have they? A quick check upstairs to make sure it carried, but the fielders are in little doubt. A pretty futile faff, the umpire's soft signal was out, and sure enough, it is!"

In [4]:
# reading the scraped data for commentary of matches from the last 3 years
df1 = pd.read_csv("/Users/murali/code/gyandata/table154.csv")
df2 = pd.read_csv("/Users/murali/code/gyandata/table155.csv")
df3 = pd.read_csv("/Users/murali/code/gyandata/table156.csv")
df4 = pd.read_csv("/Users/murali/code/gyandata/table157.csv")
df5 = pd.read_csv("/Users/murali/code/gyandata/table158.csv")
data = pd.concat([df1,df2,df3,df4,df5])
team_names = ['England','Pakistan','New Zealand','India','Australia','Bangladesh','West Indies','Afghanistan','Sri Lanka','South Africa']
data = data[(data.Team_1.isin(team_names))&(data.Team_2.isin(team_names))]
data["Comment"] = data["Comment"].apply(lambda x: x.split(","))
data["Bowler"] = data["Comment"].apply(lambda x: x[0].split(' to ')[0]) #space is included in split parameter for names like stokes, stoinis
data['Batsman'] = data["Comment"].apply(lambda x: x[0].split(' to ')[1])
data["Comment"] = data["Comment"].apply(lambda x: x[2:])
data['Comment'] = data['Comment'].apply(lambda x: word_tokenize(''.join(x))) #the comment is now split into its words
#data["Comment"] = data["Comment"].apply(lambda x: ''.join(x).split(' '))   # alternate method

data.reset_index(inplace=True, drop=True)
data.drop("Unnamed: 0", inplace=True, axis=1)

t_f = {True:1,False:0}
#data['y'] = [1 if x =='W' else 0 for x in data['Ball_Event']]
data['y'] = data['Ball_Event'].apply(lambda x: t_f[x=='W'])
data['y'].value_counts()

0    44636
1     1240
Name: y, dtype: int64

In [45]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.remove('on')
stop_words.remove('off')
stop_words.remove('above')
stop_words.remove('below')
stop_words.remove('between')
stop_words.remove('down')
stop_words.remove('through')
stop_words.remove('with')
stop_words.remove('against')

dummy = data.copy(deep=True) #deep copying ensures that changes made in dummy arent reflected in data
def process(sentence):
    for word in sentence:
        if word in stop_words:
            sentence.remove(word)
    return sentence        
dummy['Comment'] = list(map(lambda x : process(x), dummy['Comment']))
dummy[['Comment','y']]
dummy['y'].value_counts()
#filter(lambda x: stop_words not in x)


0    44636
1     1240
Name: y, dtype: int64

In [19]:
all_words = []
for x in dummy['Comment']:
    if len(x)!=0:
        for word in x:
            if word not in ['.','!',"'s","n't",'the','a']:
                all_words.append(word.lower())


In [31]:

all_words = nltk.FreqDist(all_words)
print(all_words.most_common(50))
print(type(all_words.most_common(50)))

[('outside', 11679), ('leg', 10604), ('ball', 10469), ('back', 10136), ('length', 9345), ('short', 7469), ('full', 6872), ('off', 5501), ('cover', 4784), ('midwicket', 4525), ('point', 4409), ('away', 4402), ('square', 4245), ('good', 3819), ('middle', 3815), ('stump', 3792), ('side', 3659), ('wide', 3578), ('gets', 3515), ('deep', 3457), ('one', 3230), ('towards', 2869), ('man', 2843), ('edge', 2662), ('on', 2617), ('third', 2532), ('straight', 2498), ('across', 2479), ('bat', 2390), ('line', 2353), ('delivery', 2352), ('foot', 2336), ('single', 2236), ('goes', 2170), ('forward', 2165), ('front', 2164), ('pushed', 2064), ('bowler', 1970), ('fine', 1878), ('stumps', 1878), ('driven', 1870), ('pads', 1848), ('covers', 1838), ('drive', 1832), ('time', 1829), ('long-on', 1738), ('mid-off', 1738), ('another', 1698), ('defended', 1690), ('backward', 1615)]
<class 'list'>


In [36]:
imp_words = all_words.most_common(2000)
word_features = []
for i in thousand:
    word_features.append(i[0])
def find_features(document):
    features = {}
    for w in word_features:
        # the dictionary features contains top 1000 most frequent words as keys and a bool value 
        # as the value which indicates whether that word was present in the review 
        features[w] = (w in document)  # boolean value 
    return features


In [37]:
# documents is a list of tuples with the comment and 1/0 values for all deliveries of all matches 
documents = []
for comment,y in zip(dummy.Comment, dummy.y):
    documents.append((comment,y))

featuresets = [(find_features(comment),y) for (comment,y) in documents]

In [38]:
training_set = featuresets[:45000]
testing_set = featuresets[45000:]
clf = nltk.NaiveBayesClassifier.train(training_set)
print(nltk.classify.accuracy(clf,testing_set)*100)


96.23287671232876


In [44]:
clf.show_most_informative_features(10)

Most Informative Features
                  finger = True                1 : 0      =    100.5 : 1.0
                   catch = True                1 : 0      =     70.8 : 1.0
                  simple = True                1 : 0      =     45.7 : 1.0
                    gone = True                1 : 0      =     43.7 : 1.0
                 strikes = True                1 : 0      =     36.7 : 1.0
                    what = True                1 : 0      =     35.9 : 1.0
                   bails = True                1 : 0      =     32.7 : 1.0
                decision = True                1 : 0      =     31.2 : 1.0
                 replays = True                1 : 0      =     30.9 : 1.0
                   taken = True                1 : 0      =     29.1 : 1.0
